In [1]:
""" 
1. Использовать dataset MovieLens
2. Построить рекомендации ( предсказываем оценку)
на фичах:
■ TF IDF на тегах и жанрах
■ Средние оценки median, variance, etc.) пользователя и фильма
3. Оценить RMSE на тестовой выборке
"""
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm  #  интеллектуальный индикатор выполнения - 
                                # просто оберните любую итерацию с помощью tqdm (iterable)

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline




In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [6]:
# тегов много: заменим редкие на постоянное значение 
tags.tag.nunique()

1589

In [7]:
# Заменим низкочастотные теги на значение other 

# Сделаем это так:
# Создадим отдельный Series-объект (назовем vc) с value_counts
# Отберем из него элементы с нужной частотой
# Используем индекс этих элементов - именно в индексе будет список нужных тегов (а в значениях там - число повторений тегов)
# Отфильтруем колонку с тегами по вхождению элементов столбца в список выше найденных редких тегов
# Присвоим нафильтрованным строкам в колонку с тегами наш новый тег

vc = tags['tag'].value_counts()
tags.loc[tags['tag'].isin(vc[vc < 10].index), 'tag'] = 'other'

In [8]:
# Проверим
rslt_df = tags.loc[tags['tag'] == 'other']
rslt_df.head()

,userId,movieId,tag,timestamp
1,2,60756,other,1445714996
2,2,60756,other,1445714992
3,2,89774,other,1445715207
4,2,89774,other,1445715200
5,2,89774,other,1445715205


In [9]:
# проверим количество уникальных значений тегов теперь; 
# при необходимости откалибруем фильтр частотности 2-мя ячейками выше (меняя частоту повторяемости 1500 в любую сторону)
tags.tag.nunique()

60

In [10]:
# некоторе теги встречаются заметно чаще остальных: это значит, что метод tfidf способен разнести веса от высоких к низким
# и тем не менее, вектор тегов слишком велик (соответствует всем тегам для каждого фильма + нулевые отметки, если 
# данный тег фильму не выставлялся)
tags['tag'].value_counts()

other                 2698
In Netflix queue       131
atmospheric             36
thought-provoking       24
superhero               24
funny                   23
surreal                 23
Disney                  23
religion                22
psychology              21
sci-fi                  21
dark comedy             21
quirky                  21
suspense                20
twist ending            19
crime                   19
visually appealing      19
politics                18
time travel             16
music                   16
mental illness          16
comedy                  15
aliens                  15
dark                    15
mindfuck                14
space                   14
dreamlike               14
black comedy            13
emotional               13
heist                   13
Stephen King            12
disturbing              12
Shakespeare             12
journalism              12
court                   12
anime                   12
satire                  12
a

In [11]:
# # Проверим
# rslt_df_leo = tags.loc[tags['movieId'] == '60756']
# # rslt_df_leo = tags.loc[tags['tag'] == 'Leonardo DiCaprio']
# rslt_df_leo.head()

# посмотрим, есть ли пустые значения столбца тегов: нет 
tags['tag'].isnull().any()

False

In [12]:
# Объявим функцию для работы с тегами
def change_tag_string(f):
    """
    Функция принимает строку и возвращает список слов, разденных в столбце символом пробелом, 
    которые склеивает (.join()) через пробел
    """
    return ' '.join(f.replace(' ', '').replace('-', '').split(' '))

In [13]:
# прогоним через функцию строки столбца тегов 
film_tags = [change_tag_string(g) for g in tags.tag.values]
tags['Movie_tags'] = film_tags
tags.sample(15)

,userId,movieId,tag,timestamp,Movie_tags
1101,474,364,Disney,1137180959,Disney
3188,567,104944,other,1525285975,other
2160,474,6620,other,1137205715,other
1294,474,1101,other,1138138004,other
2104,474,6286,other,1138039259,other
234,62,87430,other,1525555184,other
1163,474,647,other,1137375718,other
1485,474,1717,other,1138031824,other
1640,474,2517,Stephen King,1137373705,StephenKing
3597,599,1732,comedy,1498456268,comedy


In [14]:
# все теги в каждой группе соединим через пробел
tags_group = tags.groupby('movieId', as_index=False).agg({'Movie_tags' : lambda x: ' '.join(set(x))})
tags_group.sample(15)

,movieId,Movie_tags
35,107,other
1133,7156,Vietnam
671,3147,StephenKing
9,21,other
1363,45730,other
1280,27706,quirky darkcomedy other
781,4077,other
179,852,other
1432,70286,other
122,515,other


In [15]:
tags_group.shape

(1572, 2)

In [16]:
# Объявим функцию для работы с жанрами 
def change_string(s):
    """
    Функция принимает строку и возвращает список слов, разденных в столбце символом '|', 
    которые склеивает (.join()) через пробел
    """
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [17]:
# прогоним через функцию строки столбца жанров 
movie_genres = [change_string(g) for g in movies.genres.values]

# объявим новый столбец Movie_genres
movies['Movie_genres'] = movie_genres

In [18]:
# при помощи pd.merge соединим колонки с жанрами и тегами 
# (столбец genres заменим на модернизированный Movie_genres)
genres_tags = movies.merge(tags_group, on='movieId').drop(['genres'], axis=1)
genres_tags

,movieId,title,Movie_genres,Movie_tags
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,other
1,2,Jumanji (1995),Adventure Children Fantasy,other
2,3,Grumpier Old Men (1995),Comedy Romance,other
3,5,Father of the Bride Part II (1995),Comedy,remake other
4,7,Sabrina (1995),Comedy Romance,remake
...,...,...,...,...
1567,183611,Game Night (2018),Action Comedy Crime Horror,other funny
1568,184471,Tomb Raider (2018),Action Adventure Fantasy,other
1569,187593,Deadpool 2 (2018),Action Comedy SciFi,other
1570,187595,Solo: A Star Wars Story (2018),Action Adventure Children SciFi,other


In [19]:
# обучим модель для понимания, каким жанром принадлежит каждый их фильмов (получим вектор с жанрами для каждого фильма)
count_vect = CountVectorizer()
X_train_counts_genres = count_vect.fit_transform(genres_tags['Movie_genres'])
X_train_counts_genres.toarray()

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [20]:
# обучим модель, чтобы придать жанрам различные веса 
tfidf_transformer = TfidfTransformer()
X_train_tfidf_gen = tfidf_transformer.fit_transform(X_train_counts_genres)

# реализовав метод, видим, что вес фильмов изменился в зависимости от его жанров (чем реже жанр, тем выше вес) 
X_train_tfidf_gen.toarray()

array([[0.        , 0.39861329, 0.52164113, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.49508056, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.59650626, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.44396322, 0.4561219 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.46534105, 0.        , 0.62564122, ..., 0.        , 0.        ,
        0.        ]])

In [21]:
# обучим модель для понимания, какими тегами описывается каждый из фильмов 
# для экономии памяти не будем трансформировать разреженную матрицу в обычный массив (.toarray())
# в итоге получаем вектор тегов для каждого фильма 
X_train_tags = count_vect.fit_transform(genres_tags['Movie_tags'])

In [22]:
# обучим модель, чтобы придать тегам различные веса 
X_train_tfidf_tags = tfidf_transformer.fit_transform(X_train_tags)

In [23]:
# объединим 2 массива колонок в одну матрицу 
X_train_gen_2 = X_train_tfidf_gen.todense()         # методом .todense() из разряженной матрицы сделали обычную из модуля numpy 
X_train_tfidf_tags_2 = X_train_tfidf_tags.todense()
hs = np.hstack( (X_train_gen_2, X_train_tfidf_tags_2) )

In [69]:
tags_movie_ = tags.merge(movies, on='movieId').drop(['genres', 'timestamp'], axis=1)
tags_movie_groupby = tags_movie_.groupby('movieId')

In [70]:
movieId_as_series = tags_movie_groupby['movieId']
userId_as_matrix = tags_movie_groupby['userId']
shared_stack = np.hstack( (hs, movieId_as_series, userId_as_matrix) )

In [64]:
shared_stack.shape

(1572, 84)

In [73]:
# Выберем пользователя для обучения модели.
# Для этого посмотрим, количество тегов у фильмов, просмотренных данным пользователем. Выберем ТОП-10 пользователей.
# Пусть нашим пользователем станет user 537
user_data = tags_movie_.groupby('userId')['Movie_tags'].count().sort_values(ascending=False).head(10)
user_data
# user_data_df = pd.DataFrame(shared_stack)
# user_df = user_data_df.groupby('userId')['Movie_tags'].count().sort_values(ascending=False).head(10)
# user_data_df


userId
474    1507
567     432
62      370
599     323
477     280
424     273
537     100
125      48
357      45
318      41
Name: Movie_tags, dtype: int64

In [39]:
"""
Как здесь, я получаю выборку по конкретному пользователю и его фильмам...
"""
search_user = 537
user_films = (tags_movie_.loc[tags_movie_.userId == search_user]
        .merge(ratings, on='userId')
        .drop(['movieId_y', 'timestamp', 'tag'], axis=1))
#         .rename(columns={'movieId_x': 'movieId'}, inplace=True))
user_films.sample(15)

,userId,movieId_x,Movie_tags,title,Movie_genres,rating
2775,537,69481,other,"Hurt Locker, The (2008)",Action Drama Thriller War,1.0
4398,537,104841,other,Gravity (2013),Action SciFi IMAX,4.0
3325,537,89745,other,"Avengers, The (2012)",Action Adventure SciFi IMAX,5.0
1100,537,79132,psychology,Inception (2010),Action Crime Drama Mystery SciFi Thriller IMAX,5.0
705,537,79132,other,Inception (2010),Action Crime Drama Mystery SciFi Thriller IMAX,4.0
4637,537,105504,other,Captain Phillips (2013),Adventure Drama Thriller IMAX,5.0
562,537,2571,other,"Matrix, The (1999)",Action SciFi Thriller,5.0
569,537,2571,other,"Matrix, The (1999)",Action SciFi Thriller,4.0
1762,537,72998,other,Avatar (2009),Action Adventure SciFi IMAX,5.0
3580,537,90439,other,Margin Call (2011),Drama Thriller,5.0


In [77]:
"""
...однако если попробовать отсюда извлечь данные по пользователю, это не получится.
Так как же подготовить фрейм для обучения линейной регресии? где исправить? 
я затрудняюсь
"""
search_user_ = 537
user_df = pd.DataFrame(shared_stack)

user_df.head()

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,0.0,0.398613,0.521641,0.511277,0.282182,0.0,0.0,0.0,0.477459,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,"1010 1 1011 1 1012 1 Name: movieId, d...",1,1010 336 1011 474 1012 567 Name: user...
1,0.0,0.495081,0.0,0.635009,0.0,0.0,0.0,0.0,0.593008,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,"64 2 65 2 66 2 67 2 Name: movieId,...",2,64 62 65 62 66 62 67 474 Name: ...
2,0.0,0.0,0.0,0.0,0.643145,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,"913 3 914 3 Name: movieId, dtype: int64",3,"913 289 914 289 Name: userId, dtype: int64"
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5,"1519 5 1520 5 Name: movieId, dtype: int64",5,"1519 474 1520 474 Name: userId, dtype: i..."
4,0.0,0.0,0.0,0.0,0.643145,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7,"1521 7 Name: movieId, dtype: int64",7,"1521 474 Name: userId, dtype: int64"


In [80]:
# Получим средние оценки median, variance, etc.) по фильмам
movies_and_ratings = movies.merge(ratings, on='movieId')
movies_stat_film = movies_and_ratings.groupby('title')['rating'].agg([lambda x: np.mean(x), lambda x: np.median(x), lambda x: np.var(x), lambda x: np.std(x)])
movies_stat_film.columns = ['mean', 'median', 'variance', 'std']
movies_stat_film.sample(10)

,mean,median,variance,std
title,,,,
"NeverEnding Story II: The Next Chapter, The (1990)",2.500000,2.75,0.750000,0.866025
Dr. Dolittle 2 (2001),2.363636,2.50,0.413223,0.642824
P.S. (2004),4.250000,4.25,0.062500,0.250000
"Lawnmower Man, The (1992)",2.770833,3.00,0.853733,0.923977
"Doors, The (1991)",3.405405,3.50,0.957268,0.978401
The 5th Wave (2016),2.500000,2.50,1.000000,1.000000
I'll Do Anything (1994),2.583333,3.00,0.368056,0.606676
Money Talks (1997),3.333333,3.00,0.222222,0.471405
To Die For (1995),3.312500,3.00,0.667969,0.817294


In [81]:
# Получим средние оценки median, variance, etc.) по пользователям

movies_stat_user = movies_and_ratings.groupby('userId')['rating'].agg([lambda x: np.mean(x), lambda x: np.median(x), lambda x: np.var(x), lambda x: np.std(x)])
movies_stat_user.columns = ['mean', 'median', 'variance', 'std']
movies_stat_user.sample(10)

,mean,median,variance,std
userId,,,,
174,3.656716,3.0,0.971709,0.985753
238,3.681818,4.0,0.716942,0.846724
479,3.320442,3.0,1.411129,1.187910
520,3.881720,4.0,0.666117,0.816160
231,3.854167,4.0,0.864149,0.929596
27,3.548148,4.0,1.121756,1.059130
538,4.472973,4.5,0.337107,0.580609
280,3.900510,4.0,0.552602,0.743372
428,2.640000,3.0,1.003733,1.001865
